# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.01,83,79,1.34,NZ,1699675237
1,1,hamilton,39.1834,-84.5333,8.38,58,0,0.89,US,1699675238
2,2,port-aux-francais,-49.3500,70.2167,3.16,82,100,13.64,TF,1699675238
3,3,bethel,41.3712,-73.4140,6.94,76,10,0.45,US,1699675239
4,4,isafjordur,66.0755,-23.1240,1.88,67,78,2.24,IS,1699675240


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", size="Humidity", alpha=0.5, color="City", geo=True, tiles="OSM", height=900, width=1200)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Cloudiness'] == 0)  & (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)  & (city_data_df['Wind Speed'] < 4.5) ,:]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,tufanganj,26.3167,89.6667,24.57,56,0,2.43,IN,1699675241
16,16,taoudenni,22.6783,-3.9836,22.26,19,0,4.18,ML,1699675244
40,40,papatowai,-46.5619,169.4708,21.01,30,0,0.52,NZ,1699675251
81,81,mancio lima,-7.6142,-72.8958,25.04,94,0,2.06,BR,1699675258
196,196,porto velho,-8.7619,-63.9039,26.03,89,0,1.03,BR,1699675121
214,214,tura,25.5198,90.2201,25.19,61,0,0.23,IN,1699675286
260,260,benghazi,32.1167,20.0667,21.61,54,0,4.37,LY,1699675294
303,303,patos,-7.0244,-37.2800,24.55,61,0,2.67,BR,1699675253
373,373,luxor,25.6989,32.6421,25.03,50,0,1.54,EG,1699675314
404,404,mariani,26.6667,94.3333,24.07,69,0,1.88,IN,1699675318


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = ideal_cities_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df.insert(column='Hotel Name',loc=5,value='')

# Display sample data
hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,tufanganj,IN,26.3167,89.6667,56,
16,taoudenni,ML,22.6783,-3.9836,19,
40,papatowai,NZ,-46.5619,169.4708,30,
81,mancio lima,BR,-7.6142,-72.8958,94,
196,porto velho,BR,-8.7619,-63.9039,89,
214,tura,IN,25.5198,90.2201,61,
260,benghazi,LY,32.1167,20.0667,54,
303,patos,BR,-7.0244,-37.2800,61,
373,luxor,EG,25.6989,32.6421,50,
404,mariani,IN,26.6667,94.3333,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

# set up a parameters dictionary
params = {
    "categories": 'accommodation.hotel',
    "limit": 20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
tufanganj - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
mancio lima - nearest hotel: No hotel found
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
tura - nearest hotel: No hotel found
benghazi - nearest hotel: ترجاب
patos - nearest hotel: Hotel Zurique
luxor - nearest hotel: Salam
mariani - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
tazacorte - nearest hotel: App Leyma
atar - nearest hotel: فندق سكليل
morondava - nearest hotel: Hotel Sharon Menabe
san agustin de valle fertil - nearest hotel: Cabañas Valle Pintado


,City,Country,Lat,Lng,Humidity,Hotel Name
5,tufanganj,IN,26.3167,89.6667,56,No hotel found
16,taoudenni,ML,22.6783,-3.9836,19,No hotel found
40,papatowai,NZ,-46.5619,169.4708,30,No hotel found
81,mancio lima,BR,-7.6142,-72.8958,94,No hotel found
196,porto velho,BR,-8.7619,-63.9039,89,Hotel Slaviero Essential Porto Velho
214,tura,IN,25.5198,90.2201,61,No hotel found
260,benghazi,LY,32.1167,20.0667,54,ترجاب
303,patos,BR,-7.0244,-37.2800,61,Hotel Zurique
373,luxor,EG,25.6989,32.6421,50,Salam
404,mariani,IN,26.6667,94.3333,69,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotels_df.hvplot.points("Lng", "Lat", size="Humidity", alpha=0.5, color="City", geo=True, tiles="OSM", height=900, width=1200, hover_cols=['Hotel Name', 'Country'])

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)